# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
weather_path = "output_data\cities.csv"
weather_data = pd.read_csv(weather_path)
weather_data.head()

,Id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,George Town,5.41,100.34,78.80,78,20,3.36,MY,1609451138
1,2,Clyde River,70.47,-68.59,-4.00,83,90,9.17,CA,1609451142
2,3,East London,-33.02,27.91,69.33,79,74,15.26,ZA,1609451146
3,4,Ribeira Grande,38.52,-28.70,59.00,87,75,19.46,PT,1609451150
4,5,Saint-Paul,-21.01,55.27,73.40,83,20,13.87,RE,1609451155


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
gmaps.configure(api_key=g_key)
city_locations = weather_data[['Lat', 'Lng']]
humidity=weather_data["Humidity"].astype(float)

In [56]:
fig=gmaps.figure(center=(40.21, -3.71), zoom_level=2.2)
heat_layer=gmaps.heatmap_layer(city_locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
criteria_weather=weather_data[(weather_data["Max Temp"]>70)&(weather_data["Max Temp"]<80)&(weather_data["Wind Speed"]<10)&(weather_data["Cloudiness"]==0)]
criteria_weather

,Id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,37,Caravelas,-17.71,-39.25,75.65,77,0,7.31,BR,1609451309
111,112,Maiduguri,11.85,13.16,73.76,23,0,9.28,NG,1609451673
173,174,Araouane,18.90,-3.53,75.38,27,0,2.86,ML,1609451941
274,275,Mmathubudukwane,-24.60,26.43,75.20,64,0,3.36,BW,1609452412
376,377,Rio Grande,-32.03,-52.10,72.00,53,0,1.01,BR,1609452874
416,417,Arlit,18.74,7.39,71.33,19,0,6.38,NE,1609453066
453,454,Broken Hill,-31.95,141.43,75.20,57,0,8.05,AU,1609453238
489,490,Umluj,25.02,37.27,71.96,68,0,6.38,SA,1609453394
500,501,Bonthe,7.53,-12.51,79.32,84,0,7.56,SL,1609453444
524,525,Oussouye,12.48,-16.55,78.80,88,0,2.24,SN,1609453553


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [58]:
hotel_df=criteria_weather[["Id", "Lat", "Lng", "City", "Country"]].copy()
hotel_df["Hotel Name"]=np.nan
hotel_df

,Id,Lat,Lng,City,Country,Hotel Name
36,37,-17.71,-39.25,Caravelas,BR,NaN
111,112,11.85,13.16,Maiduguri,NG,NaN
173,174,18.90,-3.53,Araouane,ML,NaN
274,275,-24.60,26.43,Mmathubudukwane,BW,NaN
376,377,-32.03,-52.10,Rio Grande,BR,NaN
416,417,18.74,7.39,Arlit,NE,NaN
453,454,-31.95,141.43,Broken Hill,AU,NaN
489,490,25.02,37.27,Umluj,SA,NaN
500,501,7.53,-12.51,Bonthe,SL,NaN
524,525,12.48,-16.55,Oussouye,SN,NaN


In [59]:
for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    ln=row["Lng"]
    search=f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{ln}&radius=5000&keyword=hotel&key={g_key}"
    
    response_hotel=requests.get(search).json()
    if response_hotel["status"]=="ZERO_RESULTS":
        print(f"No hotel found in 5000 meter radius of {row['City']}....")
        print("--------------------------------------------------")
    elif response_hotel["status"]=="OK":
        hotel_name=response_hotel["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"]=hotel_name
        print(f"{hotel_name} found in {row['City']}")
        print("--------------------------------------------------")
hotel_df=hotel_df.dropna()
hotel_df

Pousada Liberdade found in Caravelas
--------------------------------------------------
Nanne and Boi Suites found in Maiduguri
--------------------------------------------------
No hotel found in 5000 meter radius of Araouane....
--------------------------------------------------
Gill Sleepover Guesthouse found in Mmathubudukwane
--------------------------------------------------
Laghetto Viverone Rio Grande found in Rio Grande
--------------------------------------------------
Hôtel Telwa Bungalow found in Arlit
--------------------------------------------------
Royal Exchange Hotel found in Broken Hill
--------------------------------------------------
HP Red Sea Hotel found in Umluj
--------------------------------------------------
Bonthe Holiday Village found in Bonthe
--------------------------------------------------
Auberge du Routard found in Oussouye
--------------------------------------------------


,Id,Lat,Lng,City,Country,Hotel Name
36,37,-17.71,-39.25,Caravelas,BR,Pousada Liberdade
111,112,11.85,13.16,Maiduguri,NG,Nanne and Boi Suites
274,275,-24.60,26.43,Mmathubudukwane,BW,Gill Sleepover Guesthouse
376,377,-32.03,-52.10,Rio Grande,BR,Laghetto Viverone Rio Grande
416,417,18.74,7.39,Arlit,NE,Hôtel Telwa Bungalow
453,454,-31.95,141.43,Broken Hill,AU,Royal Exchange Hotel
489,490,25.02,37.27,Umluj,SA,HP Red Sea Hotel
500,501,7.53,-12.51,Bonthe,SL,Bonthe Holiday Village
524,525,12.48,-16.55,Oussouye,SN,Auberge du Routard


In [60]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [61]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))